<br>

# Introdução

In [ ]:
#!pip3 install PyPDF2

In [ ]:
import os
import re
import shutil
from PyPDF2 import PdfFileMerger, PdfFileReader

<br>

## Paths

In [ ]:
# Paths
data_path = os.path.join('..', 'data')
input_path = os.path.join(data_path, 'input')
output_path = os.path.join(data_path, 'output')
output_apartados_path = os.path.join(output_path, 'apartados')
output_final_path = os.path.join(output_path, 'final')

# Make Directories
os.makedirs(input_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)
os.makedirs(output_apartados_path, exist_ok=True)
os.makedirs(output_final_path, exist_ok=True)

<br>

## Função

Função que extrai todo o conteúdo que se encontra entre as palabras "pag " e ").pdf".
Necessária para obter o número da primeira página do documento.

In [ ]:
def extract_text(filename):
    """
    Função que pega o nome do arquivo e cria o nome nome, utilizando
    a primeira página dos documentos extraidos dos documentos do e-SAJ
    imediatamente após o "(pag "...
    """
    # Prefixo e Sufixo da busca, para ser deletado
    start = 'pag '
    end = '.pdf'
    
    # Deleta
    text = re.search('(?<={}).*?(?={})'.format(start, end), filename)
    text = text.group()
    text = text.replace(')', '')
    page_1st = text.split(' - ')[0]
    page_1st = int(page_1st)
    return '{} - {}'.format(page_1st, filename)

for i in ['Documento 25 (pag 1249 - 1251).pdf', 'Administrativa (pag 2641).pdf']:
    print(extract_text(i))

<br>

# Principal

In [ ]:
for path, dirs, files in os.walk(input_path):
    for file in files:
        input_file = os.path.join(path, file)
        output_filename = extract_text(file)
        output_file = os.path.join(output_apartados_path, output_filename)
        print(input_file)
        shutil.copy(input_file, output_file)

<br>

# Merge PDFs

In [ ]:
# List Files
list_files = os.listdir(output_apartados_path)

# Function to sort
def get_int(name):
    num = name.split(' - ')[0]
    return int(num)

# Sort
list_files.sort(key=get_int)
#list_files

In [ ]:
def adjust_bookmark(filename):
    """
    Cria um nome para o bookmark a partir do nome do arquivo.

    """
    bookmark = filename.split(' - ', maxsplit=1)[-1]
    bookmark = bookmark.replace('.pdf', '')
    return bookmark

adjust_bookmark('208 - Documento 4 (pag 208 - 209).pdf')

In [ ]:
# Call the PdfFileMerger
merged_object = PdfFileMerger()

# Append PDF files sorted
for file in list_files:
    print(file)
    bookmark = adjust_bookmark(file)
    merged_object.append(
        PdfFileReader(os.path.join(output_apartados_path, file), 'rb'),
        bookmark,
    )

# Write all the files into a file which is named as shown below
merged_object.write(os.path.join(output_final_path, 'processo.pdf'))